In [87]:
import torch
from torch import nn

In [88]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cpu


In [89]:
from sklearn import datasets

wine = datasets.load_wine()
data = wine.data
target = wine.target

print(data.shape, target.shape)
print(wine.feature_names, wine.target_names)

(178, 13) (178,)
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'] ['class_0' 'class_1' 'class_2']


In [90]:
class WineClassfier(nn.Module):

    def __init__(self, input_size, hidden_size, out_size):
        super(WineClassfier, self).__init__()

        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.out = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()

    def forward (self, X):

        feature = self.relu(self.hidden(X))
        output = self.softmax(self.out(feature))

        return output
    
input_size = data.shape[1]
hidden_size = 32 # numero escolhido arbitrariamente
out_size = len(wine.target_names) # numero de classes

net = WineClassfier(input_size, hidden_size, out_size).to(device) # cast na GPU

In [91]:
print(net)

WineClassfier(
  (hidden): Linear(in_features=13, out_features=32, bias=True)
  (relu): ReLU()
  (out): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=None)
)


In [92]:
criterion = nn.CrossEntropyLoss().to(device) # cast na GPU

In [93]:
Xtensor = torch.from_numpy(data).float()
Ytensor = torch.from_numpy(target)

# Cast na GPU
Xtensor = Xtensor.to(device)
Ytensor = Ytensor.to(device)

print(Xtensor.dtype, Ytensor.dtype)

torch.float32 torch.int64


In [94]:
# pred = net(Xtensor[0].unsqueeze(0))

# print(Xtensor.unsqueeze(0).shape)
# print(pred.shape)
# print(pred)

pred = net(Xtensor)

In [95]:
print(pred.shape, Ytensor.shape) # as dimensionalidades nao batem! Mas isso nao importa para a cross entropy

print(pred[0].data, Ytensor[0].data) # a nossa pred eh uma distribuicao de prob e o rotulo eh um unico valor

torch.Size([178, 3]) torch.Size([178])
tensor([2.1126e-36, 1.0000e+00, 0.0000e+00]) tensor(0)


In [96]:
loss = criterion(pred, Ytensor) # como nao especificamos qual indice, ele da uma media geral. ao especificar um range, recebemos a media deste range
print(loss)

tensor(1.1526, grad_fn=<NllLossBackward0>)


In [97]:
# Regressao
from sklearn import datasets

diabetes = datasets.load_diabetes()
data = diabetes.data
target = diabetes.target

print(data.shape, target.shape)
print(data[0])
print(target[0]) # agora sao valores continuos

(442, 10) (442,)
[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
 -0.04340085 -0.00259226  0.01990749 -0.01764613]
151.0


In [98]:
class WineClassfier(nn.Module):

    def __init__(self, input_size, hidden_size, out_size):
        super(WineClassfier, self).__init__()

        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.out = nn.Linear(hidden_size, out_size)
        self.softmax = nn.Softmax()

    def forward (self, X):

        feature = self.relu(self.hidden(X))
        output = self.softmax(self.out(feature))

        return output
    
input_size = data.shape[1]
hidden_size = 32 # numero escolhido arbitrariamente
out_size = 1 # Progressao da diabetes

net = WineClassfier(input_size, hidden_size, out_size).to(device) # cast na GPU

In [106]:
criterion = nn.MSELoss().to(device)

# Exercicio
# criterion = nn.L1Loss().to(device)

# Cast na GPU
Xtensor = torch.from_numpy(data).float().to(device)
Ytensor = torch.from_numpy(target).float().to(device)

print(Xtensor.shape, Ytensor.shape)

torch.Size([442, 10]) torch.Size([442])


In [107]:
# Como estamos usando MSE, tanto a pred quanto o rotulo devem ter a mesma dimensionalidade
pred = net(Xtensor)
print(pred.shape)

loss = criterion(pred.squeeze(), Ytensor)
print(loss.data)

torch.Size([442, 1])
tensor(28771.2148)


* **Revisar: Distância absoluta (L1) Loss, Mean Squared Error (MSE) Loss e CrossEntropyLoss**